# 1. 서울시 구별 CCTV 데이터와 인구 합치기

In [1]:
import numpy as np
import pandas as pd

In [2]:
# csv file => ,로 구분하여 ,가 있는 데이터를 보호하기 위해 ""를 쓴다. 
            # tab키를 사용해서 각 데이터를 보호한다. 

## 2. CCTV 데이터


In [9]:
cctv= pd.read_csv('서울시CCTV설치운영현황(자치구)_년도별_210731기준.csv',
                    skiprows=1, encoding='EUC-KR')
cctv.head()        

,구분,총계,2012년 이전,2012년,2013년,2014년,2015년,2016년,2017년,2018년,2019년,2020년,2021년
0,계,"77,032","7,667","2,200","3,491","4,439","6,582","8,129","9,947","9,876","11,961","11,132","1,608"
1,종로구,"1,772",813,0,0,210,150,1,261,85,9,200,43
2,중 구,"2,333",16,114,87,77,236,240,372,386,155,361,289
3,용산구,"2,383",34,71,234,125,221,298,351,125,307,617,0
4,성동구,"3,602",448,125,212,105,339,310,874,390,262,461,76


In [10]:
# Null data가 있는지 확인
cctv.isna().sum().sum()

0

In [11]:
# 1000 단위 구분기호 없애기 & 정수 변환
for column in cctv.columns[1:]:
    cctv[column] = cctv[column].apply(lambda x: int(x.replace(',','')))

cctv.head()

,구분,총계,2012년 이전,2012년,2013년,2014년,2015년,2016년,2017년,2018년,2019년,2020년,2021년
0,계,77032,7667,2200,3491,4439,6582,8129,9947,9876,11961,11132,1608
1,종로구,1772,813,0,0,210,150,1,261,85,9,200,43
2,중 구,2333,16,114,87,77,236,240,372,386,155,361,289
3,용산구,2383,34,71,234,125,221,298,351,125,307,617,0
4,성동구,3602,448,125,212,105,339,310,874,390,262,461,76


In [13]:
# 최근 3개년 증가율 
    # 신규 컬럼을 만들 때 또는 숫자가 포함된 컬럼명에는 ''을 함께 사용해야 한다.
cctv['최근3개년'] = cctv['2018년'] + cctv['2019년'] + cctv['2020년']
cctv['2017년이전'] = cctv.총계 - cctv['최근3개년'] - cctv['2021년']
cctv['최근증가율'] = (cctv['최근3개년'] / cctv['2017년이전'] * 100).round(2)
cctv.head(3)

,구분,총계,2012년 이전,2012년,2013년,2014년,2015년,2016년,2017년,2018년,2019년,2020년,2021년,최근3개년,2017년이전,최근증가율
0,계,77032,7667,2200,3491,4439,6582,8129,9947,9876,11961,11132,1608,32969,42455,77.66
1,종로구,1772,813,0,0,210,150,1,261,85,9,200,43,294,1435,20.49
2,중 구,2333,16,114,87,77,236,240,372,386,155,361,289,902,1142,78.98


In [14]:
cctv['최근증가율2'] = (cctv.iloc[:, 9:12].sum(axis=1) / cctv.iloc[:, 2:9].sum(axis=1) * 100).round(2)
cctv.head()

,구분,총계,2012년 이전,2012년,2013년,2014년,2015년,2016년,2017년,2018년,2019년,2020년,2021년,최근3개년,2017년이전,최근증가율,최근증가율2
0,계,77032,7667,2200,3491,4439,6582,8129,9947,9876,11961,11132,1608,32969,42455,77.66,77.66
1,종로구,1772,813,0,0,210,150,1,261,85,9,200,43,294,1435,20.49,20.49
2,중 구,2333,16,114,87,77,236,240,372,386,155,361,289,902,1142,78.98,78.98
3,용산구,2383,34,71,234,125,221,298,351,125,307,617,0,1049,1334,78.64,78.64
4,성동구,3602,448,125,212,105,339,310,874,390,262,461,76,1113,2413,46.13,46.13


In [15]:
# selection - 필요한 컬럼 추출
cctv = cctv[['구분','총계','최근증가율']]
cctv.head()

,구분,총계,최근증가율
0,계,77032,77.66
1,종로구,1772,20.49
2,중 구,2333,78.98
3,용산구,2383,78.64
4,성동구,3602,46.13


In [16]:
# 컬럼명 변경
cctv.rename(columns={'구분':'구별', '총계':'CCTV댓수'}, inplace=True)
cctv.head()

,구별,CCTV댓수,최근증가율
0,계,77032,77.66
1,종로구,1772,20.49
2,중 구,2333,78.98
3,용산구,2383,78.64
4,성동구,3602,46.13


In [17]:
# 첫번째 행(계) 삭제
    # drop(index=0. inplace=True) 
cctv.drop([0], inplace=True) 
cctv.head()


,구별,CCTV댓수,최근증가율
1,종로구,1772,20.49
2,중 구,2333,78.98
3,용산구,2383,78.64
4,성동구,3602,46.13
5,광진구,2588,112.29


In [18]:
    # 참고 : https://dojang.io/mod/page/view.php?id=2345 
    #   - 30.1 위치 인수와 리스트 언패킹 사용하기 
print(1,2,3,4,5,6,7,8,9, sep=', ') # 1 ~ 9 : positional argument, sep=', ' : variable argument

1, 2, 3, 4, 5, 6, 7, 8, 9


In [19]:
def func(name, age, addr):
    print(name, age, addr)

In [20]:
func('마리아', 25, '서울시 영등포구 양평동')

마리아 25 서울시 영등포구 양평동


In [23]:
func(name='마리아', age=25, addr='서울시 영등포구 양평동')

마리아 25 서울시 영등포구 양평동


In [21]:
def func1(name, age, addr='양평역'):
    print(name, age, addr)

In [22]:
func1('마리아', 25) # addr이 없다면, 미리 세팅된 인수가 들어감 /// addr이 있다면 기재된 항목이 들어감
                    # 위치인수와 키워드 인수는 가변적으로 사용할 수 있다.

마리아 25 양평역


In [ ]:
# 추가 개념 : polumorphism(다형성) - overriding, overloading
    # - overriding : 상속을 받은 다음에 메소드를 재정의 => python
       # 예시
        from sklearn.base import BaseEstimator  # 1. sklearn에서 BaseExtimator을 상속받아

        class MyClassifier(BaseEstimator): 
            def fit(self, X, y):                # 2. self, X, y를 정의함
                pass                
            def predict(self,X):
                pass
            def score(self, X, y):               # 상속은 부모가 만든 attribute와 method를 자식이 그대로 사용할 수 있다. 단, 그대로 사용하지 않아도 된다 == overriding
                pass                             
    # - overloading: 동일한 함수 개념을 여러 argument을 받을 수 있다.

In [25]:
# 시작하는 첫 번째 줄에 """ """(더블 코트)를 사용한다면, 함수를 만들고 사용한다는 의미
def func3(a,b): 
    """이곳은 함수에 대한 설명을 적는 부분입니다.
    인수 a는 어쩌구 저쩌구, b는 이렇고 저렇고"""
    pass

In [ ]:
func3()